# Ejemplo de Uso del Sistema de Análisis de Vibraciones

Este notebook muestra cómo utilizar el sistema de análisis de vibraciones para:
1. Cargar datos de vibración
2. Extraer características
3. Preparar datasets para entrenamiento

In [1]:
import sys
import os
import numpy as np
import pandas as pd

# Asegurarse de que el paquete vibration_analysis esté en el path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from vibration_analysis.data.loader import DataLoader
from vibration_analysis.features.time_domain import TimeFeatureExtractor
from vibration_analysis.features.frequency_domain import FrequencyFeatureExtractor

## 1. Configuración de Rutas

In [ ]:
# Definir rutas de datos
BASE_PATH = os.path.join(project_root, 'data')
print(f"Ruta base de datos: {BASE_PATH}")

# Carpetas de datos normales y anómalos (ajusta según tu estructura)
normal_folders = ['normales/2000_1', 'normales/2000_2']
anomaly_folders = ['anormales/2000_2', 'anormales/2500_3']

# Verificar existencia de carpetas
def check_folders(base_path, folder_list):
    for folder in folder_list:
        full_path = os.path.join(base_path, folder)
        if not os.path.exists(full_path):
            print(f"Advertencia: La carpeta {full_path} no existe")
        else:
            print(f"OK: {full_path}")

print("\nVerificando carpetas normales:")
check_folders(BASE_PATH, normal_folders)
print("\nVerificando carpetas anómalas:")
check_folders(BASE_PATH, anomaly_folders)

## 2. Carga de Datos

In [5]:
# Inicializar el cargador de datos
data_loader = DataLoader(BASE_PATH)

# Cargar y preparar el dataset
# Para el caso de proponer un "clasificador", no es lo óptimo para detectores de anomalías (sesgos)
dataset = data_loader.prepare_dataset(
    normal_folders=normal_folders,
    anomaly_folders=anomaly_folders,
    test_size=0.2
)

print(f"Forma del conjunto de entrenamiento: {dataset['X_train'].shape}")
print(f"Forma del conjunto de prueba: {dataset['X_test'].shape}")

Forma del conjunto de entrenamiento: (104, 780, 3)
Forma del conjunto de prueba: (27, 780, 3)


## 3. Extracción de Características

In [6]:
# Inicializar extractores
time_extractor = TimeFeatureExtractor()
freq_extractor = FrequencyFeatureExtractor(fs=1600)  # Ajustar fs según tus datos

# Función para extraer características de un conjunto de ventanas
def extract_features_from_windows(windows):
    features_list = []
    
    for window in windows:
        # Extraer características temporales y frecuenciales
        time_features = time_extractor.extract(window)
        freq_features = freq_extractor.extract(window)
        
        # Combinar características
        all_features = {**time_features, **freq_features}
        features_list.append(all_features)
    
    return pd.DataFrame(features_list)

# Extraer características
X_train_features = extract_features_from_windows(dataset['X_train'])
X_test_features = extract_features_from_windows(dataset['X_test'])

print("\nCaracterísticas extraídas:")
print(X_train_features.columns.tolist())


Características extraídas:
['std_x', 'var_x', 'rms_x', 'peak_x', 'peak_to_peak_x', 'kurtosis_x', 'skewness_x', 'crest_factor_x', 'std_y', 'var_y', 'rms_y', 'peak_y', 'peak_to_peak_y', 'kurtosis_y', 'skewness_y', 'crest_factor_y', 'std_z', 'var_z', 'rms_z', 'peak_z', 'peak_to_peak_z', 'kurtosis_z', 'skewness_z', 'crest_factor_z', 'std_combined', 'var_combined', 'rms_combined', 'peak_combined', 'peak_to_peak_combined', 'kurtosis_combined', 'skewness_combined', 'crest_factor_combined', 'spectral_centroid_x', 'spectral_spread_x', 'spectral_rms_x', 'spectral_energy_x', 'peak_frequency_x', 'mean_frequency_x', 'spectral_centroid_y', 'spectral_spread_y', 'spectral_rms_y', 'spectral_energy_y', 'peak_frequency_y', 'mean_frequency_y', 'spectral_centroid_z', 'spectral_spread_z', 'spectral_rms_z', 'spectral_energy_z', 'peak_frequency_z', 'mean_frequency_z', 'spectral_centroid_combined', 'spectral_spread_combined', 'spectral_rms_combined', 'spectral_energy_combined', 'peak_frequency_combined', 'mea

In [7]:
X_train_features

,std_x,var_x,rms_x,peak_x,peak_to_peak_x,kurtosis_x,skewness_x,crest_factor_x,std_y,var_y,...,spectral_rms_z,spectral_energy_z,peak_frequency_z,mean_frequency_z,spectral_centroid_combined,spectral_spread_combined,spectral_rms_combined,spectral_energy_combined,peak_frequency_combined,mean_frequency_combined
0,0.246631,0.060827,0.248280,0.857143,1.571429,1.056531,0.610648,3.452319,0.207306,0.042976,...,10.289511,41290.877551,53.333333,378.244125,381.860611,230.550930,7.649548,24257.170068,53.333333,381.860611
1,0.731894,0.535669,0.839568,2.425000,4.050000,-0.339946,-0.046395,2.888389,0.571844,0.327005,...,6.369775,15823.872500,34.871795,288.177775,261.245361,225.960020,14.258837,92728.755208,64.273504,261.245361
2,0.727089,0.528658,0.843884,2.222222,3.866667,-0.385152,-0.055948,2.633327,0.616205,0.379709,...,5.793477,13090.106173,90.256410,293.331895,253.423692,223.283117,14.436415,96469.839506,90.256410,253.423692
3,0.295754,0.087470,0.298441,1.333333,2.000000,1.267878,0.746055,4.467663,0.251040,0.063021,...,11.652048,52950.388889,53.333333,386.242939,380.551068,237.661330,8.973326,32903.703704,53.333333,380.551068
4,0.722504,0.522012,0.824571,2.804878,4.487805,0.199011,-0.200840,3.401621,0.657662,0.432520,...,9.185852,32908.149911,43.076923,332.833808,288.279467,244.530846,15.908169,107720.702161,71.794872,288.279467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.363253,0.131953,0.364969,3.000000,5.000000,14.340169,1.429958,8.219872,0.383994,0.147451,...,11.759953,53935.632653,53.333333,377.436766,326.814360,234.822445,10.874272,46292.564626,53.333333,326.814360
100,0.313550,0.098314,0.316565,1.333333,2.333333,0.616906,0.348506,4.211873,0.254005,0.064519,...,11.490542,51492.694444,53.333333,381.754572,382.392660,235.287879,9.112452,33667.564815,68.376068,382.392660
101,0.711950,0.506872,0.831027,2.574468,3.978723,-0.376833,-0.076569,3.097935,0.685738,0.470237,...,6.132018,14664.641467,90.256410,280.554352,247.899017,218.273644,15.053976,103939.367587,90.256410,247.899017
102,0.700016,0.490022,0.820043,2.353846,3.646154,-0.519640,0.024199,2.870394,0.671143,0.450433,...,4.688642,8573.513136,59.487179,266.960267,230.382870,207.172535,14.327527,98217.727258,73.162393,230.382870


## 4. Preparación para Entrenamiento

In [8]:
# Crear DataFrame final para entrenamiento
train_df = X_train_features.copy()
train_df['label'] = dataset['y_train']

test_df = X_test_features.copy()
test_df['label'] = dataset['y_test']

print("\nResumen del conjunto de datos:")
print(f"Muestras de entrenamiento: {len(train_df)}")
print(f"Muestras de prueba: {len(test_df)}")
print("\nDistribución de clases en entrenamiento:")
print(train_df['label'].value_counts())


Resumen del conjunto de datos:
Muestras de entrenamiento: 104
Muestras de prueba: 27

Distribución de clases en entrenamiento:
label
1    60
0    44
Name: count, dtype: int64


In [9]:
train_df

,std_x,var_x,rms_x,peak_x,peak_to_peak_x,kurtosis_x,skewness_x,crest_factor_x,std_y,var_y,...,spectral_energy_z,peak_frequency_z,mean_frequency_z,spectral_centroid_combined,spectral_spread_combined,spectral_rms_combined,spectral_energy_combined,peak_frequency_combined,mean_frequency_combined,label
0,0.246631,0.060827,0.248280,0.857143,1.571429,1.056531,0.610648,3.452319,0.207306,0.042976,...,41290.877551,53.333333,378.244125,381.860611,230.550930,7.649548,24257.170068,53.333333,381.860611,1
1,0.731894,0.535669,0.839568,2.425000,4.050000,-0.339946,-0.046395,2.888389,0.571844,0.327005,...,15823.872500,34.871795,288.177775,261.245361,225.960020,14.258837,92728.755208,64.273504,261.245361,0
2,0.727089,0.528658,0.843884,2.222222,3.866667,-0.385152,-0.055948,2.633327,0.616205,0.379709,...,13090.106173,90.256410,293.331895,253.423692,223.283117,14.436415,96469.839506,90.256410,253.423692,0
3,0.295754,0.087470,0.298441,1.333333,2.000000,1.267878,0.746055,4.467663,0.251040,0.063021,...,52950.388889,53.333333,386.242939,380.551068,237.661330,8.973326,32903.703704,53.333333,380.551068,0
4,0.722504,0.522012,0.824571,2.804878,4.487805,0.199011,-0.200840,3.401621,0.657662,0.432520,...,32908.149911,43.076923,332.833808,288.279467,244.530846,15.908169,107720.702161,71.794872,288.279467,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.363253,0.131953,0.364969,3.000000,5.000000,14.340169,1.429958,8.219872,0.383994,0.147451,...,53935.632653,53.333333,377.436766,326.814360,234.822445,10.874272,46292.564626,53.333333,326.814360,1
100,0.313550,0.098314,0.316565,1.333333,2.333333,0.616906,0.348506,4.211873,0.254005,0.064519,...,51492.694444,53.333333,381.754572,382.392660,235.287879,9.112452,33667.564815,68.376068,382.392660,0
101,0.711950,0.506872,0.831027,2.574468,3.978723,-0.376833,-0.076569,3.097935,0.685738,0.470237,...,14664.641467,90.256410,280.554352,247.899017,218.273644,15.053976,103939.367587,90.256410,247.899017,1
102,0.700016,0.490022,0.820043,2.353846,3.646154,-0.519640,0.024199,2.870394,0.671143,0.450433,...,8573.513136,59.487179,266.960267,230.382870,207.172535,14.327527,98217.727258,73.162393,230.382870,1


In [10]:
# Mostrar filas específicas del dataframe
train_df.iloc[30:40]

,std_x,var_x,rms_x,peak_x,peak_to_peak_x,kurtosis_x,skewness_x,crest_factor_x,std_y,var_y,...,spectral_energy_z,peak_frequency_z,mean_frequency_z,spectral_centroid_combined,spectral_spread_combined,spectral_rms_combined,spectral_energy_combined,peak_frequency_combined,mean_frequency_combined,label
30,0.716421,0.513259,0.815403,2.444444,4.027778,-0.286346,0.080583,2.997836,0.644634,0.415553,...,19121.926698,34.871795,293.842116,263.632862,224.382842,15.004421,100549.828189,71.794872,263.632862,0
31,0.746341,0.557025,0.844671,2.542857,4.314286,-0.390302,-0.027431,3.010470,0.622435,0.387425,...,19291.585306,34.871795,283.996951,261.354709,226.856930,15.086152,102184.269388,69.743590,261.354709,0
32,0.296263,0.087772,0.298560,1.166667,1.833333,0.238462,0.398661,3.907643,0.262638,0.068979,...,58023.722222,53.333333,386.529341,389.137644,234.209539,9.263153,35198.824074,53.333333,389.137644,0
33,0.666606,0.444363,0.782796,2.431818,4.159091,-0.103828,0.111738,3.106582,0.654236,0.428025,...,13284.176136,32.820513,316.907216,286.247626,230.405404,14.241004,92876.813533,69.743590,286.247626,0
34,0.733853,0.538540,0.778513,2.714286,5.000000,0.425117,-0.088745,3.486498,0.652413,0.425643,...,83608.964286,16.410256,269.283629,283.025600,226.939473,17.784826,125619.517007,151.794872,283.025600,1
35,0.835371,0.697844,0.902079,2.595238,4.428571,-0.161476,0.012469,2.876951,0.629229,0.395929,...,12854.014739,57.435897,266.114148,242.450580,206.689142,15.547061,115174.484505,73.846154,242.450580,1
36,0.785557,0.617099,0.833794,2.750000,5.000000,-0.071299,-0.050885,3.298177,0.649418,0.421744,...,42742.550781,18.461538,296.672044,293.678247,223.000561,16.846567,119555.095052,77.264957,293.678247,1
37,0.701629,0.492283,0.791865,2.228571,4.028571,-0.241274,0.211306,2.814334,0.684771,0.468912,...,16752.302041,90.256410,305.562443,267.793108,227.984750,15.091126,103046.017415,90.256410,267.793108,0
38,0.865573,0.749217,0.894586,2.733333,4.733333,-0.212261,-0.052640,3.055416,0.559930,0.313522,...,20769.795556,73.846154,339.178178,297.222481,225.020279,15.702873,114678.014815,80.000000,297.222481,1
39,0.723938,0.524086,0.827433,2.575758,4.151515,-0.291789,-0.120071,3.112949,0.651389,0.424307,...,19835.553719,34.871795,303.720156,270.272164,228.341951,15.178635,102747.230181,71.794872,270.272164,0
